In [30]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import joblib
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import lightgbm as lgb


from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

import eli5
from eli5.sklearn import PermutationImportance

In [31]:
train_path = '../data/train.csv'
test_path  = '../data/test.csv'
xy_path = '../data/xy_pos.csv'
rate_path = '../data/koreanbank_rate.csv'
budongsan_path = '../data/budongsan_simli.csv'
subway_path = '../data/subway_feature.csv'
rebuilding_path = '../data/rebuilding.csv'
park_path = '../data/park.csv'
school_path = '../data/elementary_XY.csv'

dt = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)
xy = pd.read_csv(xy_path)
bankrate = pd.read_csv(rate_path)
budongsan = pd.read_csv(budongsan_path)
subway_df = pd.read_csv(subway_path)
rebuilding = pd.read_csv(rebuilding_path)
park = pd.read_csv(park_path)
schools = pd.read_csv(school_path)

In [32]:
dt['is_test'] = 1
dt_test['is_test'] = 2
concat = pd.concat([dt, dt_test])

In [33]:
concat = concat.drop(['거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '주차대수', '건축면적','해제사유발생일','기타/의무/임대/임의=1/2/3/4', '단지승인일','사용허가여부', '관리비 업로드','단지신청일'], axis=1)
concat = concat.rename(columns={'전용면적(㎡)':'전용면적'})

In [34]:
concat['구'] = concat['시군구'].map(lambda x : x.split()[1])
concat['동'] = concat['시군구'].map(lambda x : x.split()[2])

concat['계약년'] = concat['계약년월'].astype('str').map(lambda x : x[:4])
concat['계약월'] = concat['계약년월'].astype('str').map(lambda x : x[4:])
del concat['계약년월']

In [35]:
#xypos 결측치 채우기(xy_pos.csv 파일 이용)
xy['주소'] = xy['시군구'] + ' ' + xy['번지']
xy = xy.drop_duplicates(['주소'])

new_data = pd.merge(concat, xy, on=['시군구','번지'], how='left', suffixes=('', '_2'))

new_data['좌표X'] = new_data['좌표X'].fillna(new_data['좌표X_2'])
new_data['좌표Y'] = new_data['좌표Y'].fillna(new_data['좌표Y_2'])

new_data = new_data.drop(['좌표X_2','좌표Y_2','주소'], axis=1)

# new_data.shape


In [36]:
#계약년/월 기준 기준금리 column 생성
new_data['계약년월'] = new_data['계약년']+'/'+new_data['계약월']
bankrate = pd.read_csv(rate_path)
bankrate = bankrate.transpose().rename_axis('계약년월').reset_index()
new_data = pd.merge(new_data, bankrate, on=['계약년월'], how='left')
new_data['금리'] = new_data[0].copy()
new_data['금리'] = new_data['금리'].fillna(0.0)
new_data.drop([0], axis=1,inplace=True)

#계약년/월 기준 부동산 매수심리 column 생성
budongsan = budongsan.transpose().rename_axis('계약년월').reset_index()
budongsan['계약년월'] = budongsan['계약년월'].str.replace('.','/')
new_data = pd.merge(new_data, budongsan, on=['계약년월'], how='left')
new_data['부동산매수심리'] = new_data[0].copy()
new_data['부동산매수심리'] = new_data['부동산매수심리'].fillna(0.0)
new_data.drop([0], axis=1,inplace=True)

new_data.drop(['계약년월'], axis=1, inplace=True)

In [37]:
#행정 자치구/동 별 평균값 정렬 및 labeling
gu_rank = new_data.groupby("구")["target"].mean().sort_values(ascending=False).index.tolist()
dong_rank = new_data.groupby("동")["target"].mean().sort_values(ascending=False).index.tolist()

gu_labeling = {gu: label for label, gu in enumerate(gu_rank)}
dong_labeling = {dong: label for label, dong in enumerate(dong_rank)}
new_data['구_labeling'] = new_data['구'].map(gu_labeling)
new_data['동_labeling'] = new_data['동'].map(dong_labeling)

In [38]:
# 번지와 동을 결합하여 '대표지번'이라는 새로운 열 추가
new_data['대표지번'] = new_data['동'] + ' ' + new_data['번지'].astype(str)
rebuilding = rebuilding.drop_duplicates(['대표지번'])

new_data = pd.merge(new_data, rebuilding[['대표지번']], on='대표지번', how='left', indicator=True)

# 재건축여부 column 생성: 매칭되면 1, 매칭되지 않으면 0
new_data['재건축여부'] = new_data['_merge'].apply(lambda x: 1 if x == 'both' else 0)
new_data = new_data.drop(columns=['_merge'], axis=1)

In [ ]:
# 공원 평가점수 (1) : 아파트 단지별 도보권 내에 위치하는 이용가능한 도시공원 정보

# 법정동 주소에서 동과 번지를 추출하여 대표지번을 생성하는 함수
def extract_representative_address(address):
    # 주소를 공백으로 분리
    parts = address.split()
    # '동'으로 끝나는 부분과 그 다음 번지 부분을 합침
    for i, part in enumerate(parts):
        if part.endswith('동'):  # '동'으로 끝나는 부분 찾기
            return part + ' ' + parts[i + 1]  # '동'과 그 다음 번지 번호를 합침
    return None  # 매칭되지 않으면 None 반환

# 새로운 열 '대표지번'에 추출된 동 번지 값 추가
park['대표지번'] = park['법정동주소'].apply(extract_representative_address)
park = park.drop_duplicates(['대표지번'])
new_data = pd.merge(new_data, park[['대표지번', '평가점수_공원']], on='대표지번', how='left')
# 결과 확인
print(new_data['평가점수_공원'].fillna('null').value_counts())

In [40]:
# 인접한 지하철역의 개수를 계산하여 파생변수를 추가하는 코드
# 지하철역의 좌표 배열 생성
subway_coords = subway_df[['위도', '경도']].values

# haversine 함수를 벡터화하여 사용
def haversine_vectorized(lat1, lon1, subway_coords):
    lat2, lon2 = subway_coords[:, 0], subway_coords[:, 1]
    R = 6371  # 지구의 반지름 (단위: km)
    
    dlat = np.radians(lat2 - lat1[:, np.newaxis])
    dlon = np.radians(lon2 - lon1[:, np.newaxis])
    
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1[:, np.newaxis])) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c  # 단위는 km

In [41]:
# 부동산 데이터의 좌표 배열 생성
property_coords = new_data[['좌표Y', '좌표X']].values

# 각 부동산 좌표와 모든 지하철역 간의 거리를 벡터화하여 계산
distances = haversine_vectorized(property_coords[:, 0], property_coords[:, 1], subway_coords)

# 반경 설정 (예: 1km)
radius_km = 1

# 반경 1km 내에 있는 지하철역 수를 계산
new_data['near_subway'] = np.sum(distances <= radius_km, axis=1)

In [42]:
# 아파트 좌표와 초등학교 좌표 추출
property_coords = new_data[['좌표Y', '좌표X']].values
school_coords = schools[['좌표Y', '좌표X']].values

# Haversine 거리 계산
distances = haversine_vectorized(property_coords[:, 0], property_coords[:, 1], school_coords)

# 각 아파트 주변 반경 내 초등학교 수 계산
new_data['near_schools'] = np.sum(distances <= radius_km, axis=1)

In [43]:
# IQR 이상치 제거 From Baseline
def remove_outliers_iqr(dt, column_name):
    df = dt.query('is_test == 1')
    df_test = dt.query('is_test == 2')

    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

    result = pd.concat([df, df_test])
    return result

In [44]:
#포지션값 이상치 제거
#결측치가 제거될거라 생각하고 사용했음 / 결측치는 모르겠는데 이상치는 제거되는듯
new_data = remove_outliers_iqr(new_data, '좌표X')
new_data = remove_outliers_iqr(new_data, '좌표Y')

#TODO: 이상치 여기서 더 제거해보면 좋을듯?

In [45]:
#5년이내 및 10년이내 건축 column 생성
new_data['new_apartment_5years'] = new_data['건축년도'].apply(lambda x: 1 if x >= 2018 else 0)
new_data['new_apartment_10years'] = new_data['건축년도'].apply(lambda x: 1 if x >= 2013 else 0)

#30년 이상되어 재건축연한 된 column 생성
new_data['over_rebuild_year'] = new_data['건축년도'].apply(lambda x: 1 if x<=1993 else 0)

#TODO: Column 추가 생성(함성변수)

In [46]:
new_data['계약년'] = new_data['계약년'].apply(pd.to_numeric)
new_data['계약월'] = new_data['계약월'].apply(pd.to_numeric)
new_data = new_data.drop(['시군구', '번지', '구', '동', '대표지번'], axis=1)

In [47]:
dt_train = new_data.query('is_test==1')
dt_test = new_data.query('is_test==2')

dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)

dt_test['target'] = 0

In [ ]:
continuous_columns = []
categorical_columns = []

for column in dt_train.columns:
    if pd.api.types.is_numeric_dtype(dt_train[column]):
        continuous_columns.append(column)
    else:
        categorical_columns.append(column)

print("연속형 변수:", continuous_columns)
print("범주형 변수:", categorical_columns)

In [ ]:
# 범주형 변수들에 대해 label 인코딩 From Baseline Code
# 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
label_encoders = {}

# Implement Label Encoding
for col in tqdm( categorical_columns ):
    lbl = LabelEncoder()

    # Label-Encoding을 fit
    lbl.fit( dt_train[col].astype(str) )
    dt_train[col] = lbl.transform(dt_train[col].astype(str))
    label_encoders[col] = lbl           # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.
    
    # Test 데이터에 새로 출현한 데이터를 신규 클래스로 추가
    for label in np.unique(dt_test[col].astype(str)):
      if label not in lbl.classes_:  # unseen label인 경우
        lbl.classes_ = np.append(lbl.classes_, label)  # ValueError를 피하기 위해 처리

    dt_test[col] = lbl.transform(dt_test[col].astype(str))

In [50]:
y_train = dt_train['target']
X_train = dt_train.drop(['target'], axis=1)

# Hold out split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2023)

In [ ]:
# 로버스트 + 로그 스케일링 적용 # 
# 타겟 변수(y)에 로그 스케일링 적용 (log1p 사용)
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

# X에 대해 로버스트 스케일링 적용 (X에만 스케일링 적용)
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)


gbm = lgb.LGBMRegressor(n_estimators=10000)

#y는 로그 변환된 상태로 학습
gbm.fit(
    X_train_scaled, y_train_log,  # 로그 변환된 y를 사용
    eval_set=[(X_train_scaled, y_train_log), (X_val_scaled, y_val_log)],  # 검증 데이터도 로그 변환
    eval_metric='rmse',
    callbacks=[lgb.log_evaluation(period=500, show_stdv=True)]
)

# 로그 스케일로 예측됨
gbm_pred_log = gbm.predict(X_val_scaled)

# 예측값에 대해 역변환 적용 (expm1 사용하여 원래 값으로 복원)
gbm_pred = np.expm1(gbm_pred_log)

# 검증 데이터에서의 RMSE 계산
rmse = np.sqrt(mean_squared_error(y_val, gbm_pred))
print(f'RMSE test: {rmse}')

In [52]:
# 로버스트 + 로그 스케일링 적용 # 

# Test dataset에 대한 inference 진행
X_test = dt_test.drop(['target'], axis=1)

# 1. Test dataset에도 동일한 스케일링 적용 (로버스트 스케일링)
X_test_scaled = scaler.transform(X_test)

# 2. Test dataset에 대한 예측 (로그 스케일로 예측된 값)
real_test_pred_log = gbm.predict(X_test_scaled)

# 3. 예측값에 대해 역변환 적용 (로그 변환을 되돌림, expm1 사용)
real_test_pred = np.expm1(real_test_pred_log)

preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
preds_df.to_csv('../outputs/output_name.csv', index=False) 